In [1]:
#%matplotlib notebook

import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage import color, img_as_ubyte
import os, sys
from scipy.spatial.distance import pdist
import csv
from skimage.transform import hough_ellipse



In [2]:

#Find centroids of bolts near edge of circles in find_pmts() using findContours()

#Output is a list of bolt locations for a given circle 
def ret_centres(image, params) :
    out = []
    x = params[0]
    y = params[1]
    r = params[2]
    L = int(r*r_mult)

    gray_image = image[y-L:y+L, x-L:x+L]
    ret,thresh = cv2.threshold(gray_image,127,255,0)
    gray_image = color.gray2rgb(img_as_ubyte(gray_image))
    

    # find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #Loop over bolts and find centroid
    for c in contours:
    #moments
        M = cv2.moments(c)
        area = cv2.contourArea(c)
        
        if (area <=2) or (area>= 500) :  # skip ellipses smaller then 10x10
            continue
            
           # calculate x,y coordinate of center, breaks if m00 = 0 output location in large image (offset)
        elif M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            out.append([cX + (x-L), cY + (y-L)])
             #   print(area)
 #           cv2.circle(gray_image,(cX, cY),10,(0,255,255),2)
        else:
            continue
    
  #  cv2.imshow('img', gray_image)
  #  cv2.waitKey(0)
    return out


#Orders bolts correctly
def number_bolts(circle, bolt_ring) :
    out = []
    circ_x = circle[0]
    circ_y = circle[1]
    circ_r = circle[2]
    L = int(circ_r*r_mult)  
      
    nodes = np.asarray(bolt_ring)
    for i in np.linspace(0, 2*3.14159, num=60) : 
        search_x = int(circ_x + L*np.sin(i))
        search_y = int(circ_y - L*np.cos(i))
        dist = np.sum((nodes - [search_x,search_y])**2, axis=1)
        closest = np.argmin(dist)
            
        if bolt_ring[closest] not in out :
            out.append(bolt_ring[closest])
    return out


#Diameters of bolts
def bolt_avg_dia(circle, bolt_ring) :
    dias = []
    num = []
    lengths = []
    num.append(len(bolt_ring))
    
    #debug

    
    
    circ_x = circle[0]
    circ_y = circle[1]
    circ_r = circle[2]
    L = int(circ_r*r_mult)          

#    test_image = image[y-L:y+L, x-L:x+L]
#    ret,thresh = cv2.threshold(test_image,127,255,0)
#    test_image = color.gray2rgb(img_as_ubyte(test_image))
    

    centre = np.asarray([circ_x, circ_y]) 
    nodes = np.asarray(bolt_ring)
    
   # print(f'centre:{centre}')
    #print(f'nodes:{nodes}')
    #print(f'bolt_ring:{bolt_ring}')
    for i in np.linspace(0, 3.14159, num=60) : 
        x_1 = int(circ_x + L*np.sin(i))
        y_1 = int(circ_y - L*np.cos(i))

        dist_1 = np.sum((nodes - [x_1,y_1])**2, axis=1)
        
            
        dia_1 = bolt_ring[np.argmin(dist_1)]
        dist_2 = np.sum((nodes - dia_1)**2, axis=1)
        
        dia_2 = bolt_ring[np.argmax(dist_2)]
        
        rad_1 = centre - dia_1
        rad_2 = centre - dia_2
        #print(i, np.argmin(dist_1), np.argmax(dist_2), dia_1)
        angle = np.arccos(np.dot(rad_1, rad_2) / (np.linalg.norm(rad_1) * np.linalg.norm(rad_2)))
        if (angle > 3) and ([dia_1, dia_2] not in dias) :
            #print(dia_1[0])
            dias.append([dia_1, dia_2])
            lengths.append(np.sqrt((dia_1[0]-dia_2[0])**2 + (dia_1[1]-dia_2[1])**2))
            #print(angle)        
    return([dias, num, lengths])
    
#plot histo of results
def eval_reco(odir, info, img) :
    #dias = info[0]
    num_bolts = info[0]
    lengths = info[1]
    
  #  print(num_bolts[0])
   ## print(lengths[0])
    #ax1 = plt.subplot(212)
    #ax1.margins(1)           # Default margin is 0.05, value 0 means fit
    #ax1.imshow(img)

    ax2 = plt.subplot(221)
    ax2.hist(num_bolts, np.arange(10,25,1), edgecolor='black', linewidth=0.75)
    ax2.set_xlabel('Number of bolts')
    ax2.set_ylabel('PMTs')
    ax2.margins(0.1)
    
    ax3 = plt.subplot(222)
    ax3.hist(lengths, edgecolor='black', linewidth=0.75)
    ax3.set_xlabel('Percentage uncertainty in diameter')
    ax3.set_ylabel('Bolt Pairs')
    ax3.margins(0.1)
    #plt.show()
    
    


In [3]:
#Approx pmt locations

def find_pmts(pmt_img, bolt_img, pmt_locs) :

    out = []

    #r = 210 
    circles = []
    ellipses = []
   # print(pmt_locs)
    r = int(0.5*np.average(np.diff( np.unique(np.array(pmt_locs['pmt_pos'])[:,1]).astype('float64') ))*r_mult) # MULTIPLIER SCALES R IT IS SLIGHTLY TOO BIG AS TAKEN FROM GAPS
    for i in range(len(pmt_locs['pmt_pos'])) :
            posn = pmt_locs['pmt_pos'][i]
            label = pmt_locs['pmt_label'][i]
            
            if posn[0] > 300 and posn[1] > 300 :
                

                #print(f'PMT: {posn} {label}')
                
                one_pmt_img = pmt_img[int(posn[0])-r:int(posn[0])+r, int(posn[1])-r:int(posn[1])+r]
             #   one_pmt_bolt_img_orig = bolt_img[int(posn[0])-r:int(posn[0])+r, int(posn[1])-r:int(posn[1])+r]
                

                #ellipse fit :
                '''
                edges = cv2.Canny(gray_pmt_img,0,100)
                print([i, j])
                result = hough_ellipse(edges, accuracy=50, threshold=100, min_size=r, max_size=2*r)
                result.sort(order='accumulator')
                if len(result) == 0 :
                    continue
                best = list(result[-1])
                yc, xc, a, b = [int(round(x)) for x in best[1:5]]
                circles.append([int(j)+200+yc, int(i)+100+xc, int(a), int(b)])
                '''
                #blob
                '''
               # detector = cv2.SimpleBlobDetector()
                # Setup SimpleBlobDetector parameters.
                params = cv2.SimpleBlobDetector_Params()
                #params.minThreshold = 250
                #params.filterByColor = True
                #params.blobColor = 0
                # Filter by Area.
                params.filterByArea = True
                params.minArea = 500
                # Filter by Circularity
                params.filterByCircularity = True
                params.minCircularity = 0.1

                # Create a detector with the parameters
                detector = cv2.SimpleBlobDetector_create(params)
                # Detect blobs.
                keypoints = detector.detect(inv_pmt_img)                
                im_with_keypoints = cv2.drawKeypoints(inv_pmt_img, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                print(keypoints)
                # Show blobs
                cv2.imshow("Keypoints", im_with_keypoints)
                cv2.waitKey(0)
                '''
                contours, hierarchy = cv2.findContours(one_pmt_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                area = 0
                coords = []
                
                #debug:draw contours
                #cont_pmt_img = cv2.drawContours(one_pmt_img, contours, -1, (255,255,0), 3)
                #cv2.imshow('img', cont_pmt_img)
                #cv2.waitKey(0)
                
                
                for c in contours:
                #moments
                    new_M = cv2.moments(c)
                    new_area = cv2.contourArea(c)

                    if (new_area <=300) or (new_area>= 100000) :  # skip ellipses smaller then 10x10
                        continue

                       # calculate x,y coordinate of center, breaks if m00 = 0 output location in large image (offset)

                    elif new_M["m00"] != 0:
                        cX = int(new_M["m10"] / new_M["m00"])
                        cY = int(new_M["m01"] / new_M["m00"])
                                 #   print(area)

                    else:
                        continue

                    if new_area >= area :    #TODO:BETTER CHECK?
                        area = new_area
                        coords = [cX, cY, r]
                       # print(area)
                    
                #Fit an ellipse  
                #cut image to center of centroid of pmt
                #print(coords)
                #TODO: posn coords are wrtong way round
                
                #TODO: Check by drawing contours
                if len(coords) > 0 :
                    x_cor_circ = int(posn[1]) - r
                    y_cor_circ = int(posn[0]) - r

                    x_cor_ell = int(coords[1]) - r
                    y_cor_ell = int(coords[0]) - r

                    #print([x_cor_circ, x_cor_ell])
                    #corrected to center image on centre of blob
                    one_pmt_bolt_img = bolt_img[int(coords[1])+y_cor_circ-r:int(coords[1])+y_cor_circ+r, int(coords[0])+x_cor_circ-r:int(coords[0])+x_cor_circ+r]

                   # cv2.imshow('bolt', one_pmt_bolt_img)
                   # cv2.imshow('pmt', one_pmt_img)
                   #' cv2.imshow('bold', one_pmt_bolt_img_orig)
                    #cv2.waitKey(0)
                

                    #print([i, j, coords, r, int(i)+100-r, int(j)+200-r])
                    circles.append([int(coords[0])  + x_cor_circ, int(coords[1]) + y_cor_circ, int(r), label])
                    ellipse_params =  fit_to_bolts(one_pmt_bolt_img, coords) #[1,1,1,1,1] #   
                    ellipses.append([ellipse_params[0]+ x_cor_circ, ellipse_params[1]+y_cor_circ, ellipse_params[2], ellipse_params[3], ellipse_params[4], label, ellipse_params[5]])

            #    circles.append([int(j)+200, int(i)+100, int(r)])
    return circles, ellipses
    

In [4]:
#Main Bolt Finder
#TODO : MARGIN AT BOTTOM OF IMAGE
def find_centroids(idir, odir, img_name, guess) :
    #pmt_guess = open(f'/home/dm3315/Documents/SK/PMT_learning/images/ring_reco/four_gaps/guess_meth/pmt_loc.txt', "r")

    feature_info = open(f'{odir}coords/{img_name}.txt', "w")
    
    img = cv2.imread(f'{idir}{img_name}.png')

    #Set up separate masks for bolts and pmts and an output image
    train_labels = img
    lower_pmts = np.array([254, 0, 0], dtype = "uint16")
    upper_pmts = np.array([255,0,0], dtype = "uint16")
    lower_bolts = np.array([0, 0, 254], dtype = "uint16")
    upper_bolts = np.array([0,0,255], dtype = "uint16")

    pmts_gray = cv2.inRange(train_labels, lower_pmts, upper_pmts)
    bolts_gray = cv2.inRange(train_labels, lower_bolts, upper_bolts)
    img = cv2.cvtColor(bolts_gray,cv2.COLOR_GRAY2BGR) #used for making output image

    #print('images made')

    #Calls HoughCircles() to find multiple PMTs in an image

    #Outpts list of circle parameters



    
    #find circles in image
    
    #TODO: Do circles then filter out bads, then do ellipse to save time
    circles, ellipses = find_pmts(pmts_gray, bolts_gray, guess) ######### TODO:DO ELLIPSE FIT IN HERE RETURNED CIRCLES ARE INSRTEAD ELLIPSES
    bolt_centres = []

    print(f'number of circles:{len(circles)}, ellipses:{len(ellipses)}')
    #loop over circles and find bolts, end up with list of circles, each a  list of bolts (no circle parameters are saved )
    bolt_no = 0
    pmt_reco = 0
    
    #info contains list of circles, each a list of bolt centres, and number of bolts
    info = []
    info.append([])
    info.append([])
    dia_err = []
    
    
    
    #output ellipse fits
    for i in range(len(circles)) :
        
        circ = circles[i]
        ell = ellipses[i]
        
       # print([[circ[0], circ[1]], [ell[0], ell[1]]])
        cv2.circle(img,(circ[0],circ[1]),circ[2],(0,100,0),2)

        bolt_ring = ret_centres(bolts_gray, circ) # list of bolt centres
        #temp_bolts_gray = bolts_gray.copy()

       # cv2.imshow('img2', img)
     #   cv2.waitKey(0)
        if len(bolt_ring) < 3 :
           # cv2.circle(img,(circ[0],circ[1]),circ[2],(0,255,0),2)
           # feature_info.write(f'00    {circ[1]}    {circ[0]}\n')
            continue
            
        bolts_ord = number_bolts(circ, bolt_ring) # ordered list of bolt centres
        dias = bolt_avg_dia(circ, bolts_ord)      #lis of bolt centres, diameters and number of bolts in circle


        avg_dia = 0
        avg_centre = 0
        num = len(dias[2])
        
      #  if num<3:
      #          cv2.circle(img,(circ[0],circ[1]),circ[2],(0,100,0),2)
      #          continue
        pmt_reco += 1
        
        #Draw diameters here
        #for j in dias[0] : 
        #    print(j, j[0])
            #avg_dia += np.linalg.norm(np.array(np.array(j[1]) - np.array(j[0]))) / num
            #avg_centre += 
           # cv2.line(img, (j[0][0], j[0][1]), (j[1][0], j[1][1]), (0, 255, 0), thickness=2, lineType=8)
          #  cv2.imshow('img', img)
           # cv2.waitKey(0)
        #Draw hough circles on img
        #if num !=0 :
            #avg_dia = sum_dia/num

            
        cv2.ellipse(img, (int(ell[0]), int(ell[1])), (int(ell[2]), int(ell[3])), int(ell[4]), 0, 360, (0, 255, 255), 2)
        cv2.putText(img, f'{circ[3]}', (int(ell[0]), int(ell[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        feature_info.write(f'00:{circ[0]}:{circ[1]}:{circ[2]}:{ell[0]}:{ell[1]}:{ell[2]}:{ell[3]}:{ell[4]}:{circ[3]}:{ell[6]}\n')

        avg_dia = np.average(dias[2])
        dia_err = np.asarray(dias[2])/avg_dia
        info[0].extend(dias[1])
        info[1].extend((dia_err - 1)*100)
        

        bolt_no = 0
    #draw bolt locations
        for k in bolts_ord :
            bolt_no += 1
#        cv2.circle(img, (j[1], j[0]), 5, (0,0,255), -1)
            cv2.putText(img, f'{bolt_no}', (k[0], k[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)
            feature_info.write(f'{bolt_no}:{k[0]}:{k[1]}:{circ[3]}\n')

    #print(info)
    #make a plot of bolt numbers and diameters in whole image
    print(f'number of pmts = {len(circles)}')
    print(f'number of pmts reconstructed = {pmt_reco}')
    print(f'percentage PMTs = {pmt_reco}')

    print(f'number of bolt pairs = {np.sum(info[0])/2}')
    print(f'number of diameters = {len(info[1])}')
    eval_reco(odir, info, img)
    cv2.imwrite(f'{odir}plots/{img_name}_labelled.png', img )
    feature_info.close()

In [5]:
#Ellipse fitter - run on small bolt image

def fit_to_bolts(img, init_coords) :


    #print(circle)
    circ_x = init_coords[0]
    circ_y = init_coords[1]
    circ_r = init_coords[2]
    L = int(circ_r)          
    
    x_cor_ell = circ_x - circ_r
    y_cor_ell = circ_y-circ_r
    
    best_param = [x_cor_ell, y_cor_ell, 10, 10, 0]
    draw_temp_img = img.copy()
    centre = np.asarray([circ_x, circ_y])

    # gray_image = bolts_gray[circ_y-L:circ_y+L, circ_x-L:circ_x+L]

   # cv2.imshow('img', img)
   # cv2.waitKey(0)
    
    #First loop fits a circle and its centre 
    score = 0
    for i in range(int(circ_r*0.4)) : #r1
        for j in range(40) : # xcen
            for k in range(40) :# ycen
                new_r1 = circ_r*0.8 + i
                new_r2 = circ_r*0.8 + i

                mult = 2

                new_cen = (L - 20*mult + j*mult, L - 20*mult + k*mult)
                new_r = (int(new_r1), int(new_r2))
                new_ang = 0
                circ_mask = np.zeros(img.shape,dtype=np.uint8)

               # print(new_cen, new_r, new_ang, 0, 360, 1, 1)
                cv2.ellipse(circ_mask, new_cen, new_r, new_ang, 0, 360, color=1 , thickness=1)
                fit = img*circ_mask
                new_score = np.sum(fit)
                
                
                if new_score >= score :
                    score = new_score
                    best_param = [new_cen[0]+x_cor_ell, new_cen[1]+y_cor_ell, new_r1, new_r2, new_ang, score ]

#############################################################################
#second loop does the ellipse 
    for j in range(int(circ_r*0.8)) : #r2
        for m in [0, 45] : # angle
                #print(i)
                new_r1 = best_param[2]
                new_r2 = circ_r*0.6+ j

                new_r = (int(new_r1), int(new_r2))
                new_ang = m
                new_cen = (best_param[0]-x_cor_ell, best_param[1]-y_cor_ell)
                circ_mask = np.zeros(img.shape,dtype=np.uint8)
                cv2.ellipse(circ_mask, new_cen, new_r, new_ang, 0, 360, 1, 1)
                fit = img*circ_mask

                #Debug
                #cv2.imshow('img', fit)
                #cv2.waitKey(0)   
                
                init_rad = circ_r
                new_score = np.sum(fit)
                if new_score >= score :
                    score = new_score
                    best_param = [new_cen[0]+x_cor_ell, new_cen[1]+y_cor_ell, new_r1, new_r2, new_ang, score ] #TODO:DIFFERENT
    #debug
    #print(best_param)
    #print(int(best_param[0]-x_cor_ell))
    #cv2.ellipse(draw_temp_img, (int(best_param[0]-x_cor_ell), int(best_param[1]-y_cor_ell)), (int(best_param[2]), int(best_param[3])), int(best_param[4]), 0, 360, (255, 255, 255), 2)
  #  cv2.imwrite(f'{odir}plots/temp/{circ_x}.png', draw_temp_img )

    return best_param



## Run below here

In [10]:
import time 
start_time = time.time()

dataset = 'substacks_far'
seg_files = 'ring_avg_all'
method = 'manual_intervention'
supervision = 'supervised'
info_filename = 'fixed_pmt_loc_label'

#img_name = 'SING0046'
#img_path = f'/home/dm3315/Documents/SK/PMT_learning/images/datasets/{dataset}/output/output/{img_name}.png'
idir = f'/home/dm3315/Documents/SK/PMT_learning/images/datasets/{dataset}/output/{seg_files}/'
odir = f'{idir}feature_info/{supervision}/'
#img_path = '/home/dm3315/Documents/SK/PMT_learning/images/datasets/bolts_pmt/output/output/large_test_out/4_pred_noverlay.png'

f = []
for (dirpath, dirnames, filenames) in os.walk(f'{idir}'):
    f.extend(filenames)
    break

#f = [x[:-4] for x in f]
#f = f[:]
f = ['794_pred_substacks_far_unet.19']
print(f)
#f = ['B7730726_pred_substacks_far_unet.19']

guess_info = {}
with open (f'/home/dm3315/Documents/SK/PMT_learning/images/datasets/{dataset}/output/{seg_files}/smodule/{method}/{info_filename}.txt', 'r') as info:
    for row in csv.reader(info,delimiter=':') :
        if str(row[0]) not in guess_info:
            guess_info[str(row[0])] = {'pmt_pos':[], 'pmt_label':[]}
        guess_info[str(row[0])]['pmt_pos'].append([float(row[1]), float(row[2])])
        guess_info[str(row[0])]['pmt_label'].append([float(row[3]), float(row[4]), float(row[5]), float(row[6])])
        
#print(guess_info)

global r_mult #for cutting windows around PMTs
r_mult = float(1.0)

for i in f :
    if i[:3] in guess_info :
        print(f'processing image: {i[:3]}')
        #print(f'{guess_info[str(i[:4])]}')
        find_centroids(idir, odir, i, guess_info[str(i[:3])])
        plt.close()
    else:
        print(f'PMT information not provided for {i[:3]}')
        
print(f'time taken = {time.time() - start_time} seconds')


cv2.destroyAllWindows()

['794_pred_substacks_far_unet.19']
processing image: 794
number of circles:54, ellipses:54


<ipython-input-2-a54089b8c529>:105: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.dot(rad_1, rad_2) / (np.linalg.norm(rad_1) * np.linalg.norm(rad_2)))


number of pmts = 54
number of pmts reconstructed = 52
percentage PMTs = 52
number of bolt pairs = 375.0
number of diameters = 202
time taken = 1310.2389328479767 seconds
